In [42]:
import pandas as pd

In [43]:
file = pd.read_csv("C:/Users/Owner/Desktop/kdramalist.csv")
file.head()

,age,gender,bmi,children,smoker,region,medical_history,family_medical_history,exercise_frequency,occupation,coverage_level,charges
0,46,male,21.45,5,yes,southeast,Diabetes,NaN,Never,Blue collar,Premium,20460.307669
1,25,female,25.38,2,yes,northwest,Diabetes,High blood pressure,Occasionally,White collar,Premium,20390.899218
2,38,male,44.88,2,yes,southwest,NaN,High blood pressure,Occasionally,Blue collar,Premium,20204.476302
3,25,male,19.89,0,no,northwest,NaN,Diabetes,Rarely,White collar,Standard,11789.029843
4,49,male,38.21,3,yes,northwest,Diabetes,High blood pressure,Rarely,White collar,Standard,19268.309838


In [44]:
df = pd.DataFrame(file)
target = df['charges']
datas = df.drop(columns=['charges'])

In [45]:
data = datas.dropna()
print(data)

        age  gender    bmi  children smoker     region      medical_history  \
1        25  female  25.38         2    yes  northwest             Diabetes   
4        49    male  38.21         3    yes  northwest             Diabetes   
6        64  female  20.12         2     no  northeast  High blood pressure   
7        53    male  30.51         4     no  southeast        Heart disease   
11       45  female  39.68         1     no  northwest  High blood pressure   
...     ...     ...    ...       ...    ...        ...                  ...   
999991   49    male  32.51         0    yes  southwest  High blood pressure   
999992   42    male  25.92         0     no  northwest  High blood pressure   
999997   39    male  39.84         0    yes  northeast        Heart disease   
999998   37  female  45.06         4    yes  northeast  High blood pressure   
999999   41    male  30.02         2    yes  northeast             Diabetes   

       family_medical_history exercise_frequency   

1. 필요한 라이브러리 import

In [46]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import warnings  # 경고 메시지 무시
warnings.filterwarnings('ignore')

# 한글 폰트를 설정하여 시각화에 한글이 깨지지 않도록 설정
plt.rcParams['font.family'] = 'gulim'

In [47]:
## data['family_medical_history'] == data['medical_history']
matching_rows = data[data['family_medical_history'] == data['medical_history']]
matching_count = len(matching_rows)
print("일치하는 행의 개수:", matching_count)

일치하는 행의 개수: 187222


In [48]:
## medical_history 튜플 별로 개수
values_to_count = ['Diabetes', 'High blood pressure', 'Heart disease']
value_counts = data['medical_history'].value_counts()

# 결과 출력
for value in values_to_count:
    print(f"{value}: {value_counts.get(value, 0)}")
print(f"NaN: {nan_count}")

Diabetes: 186957
High blood pressure: 186869
Heart disease: 187846
NaN: 0


In [52]:
##'exercise_frequency' 열로 그룹화하고 'bmi' 열을 분류
bmi_counts= datao.groupby('exercise_frequency')['bmi'].value_counts()

# 결과 출력
print(bmi_counts)

exercise_frequency  bmi  
Frequently          49.43    69
                    28.90    68
                    32.99    68
                    48.38    67
                    49.18    66
                             ..
Rarely              43.44    23
                    23.56    22
                    44.65    22
                    49.24    22
                    18.00    17
Name: count, Length: 12804, dtype: int64


In [49]:
##  bmi >= 25 그리고 medical_history == 'High blood pressure'
filtered_data = data[(data['bmi'] >= 25) & (data['medical_history'] == 'High blood pressure')]
filtered_columns1 = filtered_data[['bmi', 'medical_history']]

print(filtered_columns1)

count = len(filtered_columns1)

# 결과 출력
print(f"조건: bmi >= 25 그리고 medical_history == 'High blood pressure': {count}")

          bmi      medical_history
11      39.68  High blood pressure
13      39.95  High blood pressure
22      31.11  High blood pressure
28      26.46  High blood pressure
29      30.75  High blood pressure
...       ...                  ...
999977  32.26  High blood pressure
999985  37.00  High blood pressure
999991  32.51  High blood pressure
999992  25.92  High blood pressure
999998  45.06  High blood pressure

[146046 rows x 2 columns]
조건: bmi >= 25 그리고 medical_history == 'High blood pressure': 146046


In [50]:
## bmi >= 25 그리고 medical_history == 'Heart disease'
filtered_data = data[(data['bmi'] >= 25) & (data['medical_history'] == 'Heart disease')]
filtered_columns2 = filtered_data[['bmi', 'medical_history']]

print(filtered_columns2)

count = len(filtered_columns2)

# 결과 출력
print(f"조건: bmi >= 25 그리고 medical_history == 'Heart disease': {count}")

          bmi medical_history
7       30.51   Heart disease
26      39.70   Heart disease
30      41.15   Heart disease
46      38.53   Heart disease
49      40.31   Heart disease
...       ...             ...
999957  45.03   Heart disease
999958  34.19   Heart disease
999960  26.38   Heart disease
999978  40.60   Heart disease
999997  39.84   Heart disease

[146856 rows x 2 columns]
조건: bmi >= 25 그리고 medical_history == 'Heart disease': 146856


In [51]:
## 'exercise_frequency' 열로 그룹화하고 'medical_history' 열을 분류
exercisemedicalhistory_counts= data.groupby('exercise_frequency')['medical_history'].value_counts()

# 결과 출력
print(exercisemedicalhistory_counts)

## 필요없을듯

exercise_frequency  medical_history    
Frequently          Heart disease          46883
                    High blood pressure    46781
                    Diabetes               46541
Never               High blood pressure    46857
                    Diabetes               46792
                    Heart disease          46636
Occasionally        Heart disease          47207
                    High blood pressure    46763
                    Diabetes               46645
Rarely              Heart disease          47120
                    Diabetes               46979
                    High blood pressure    46468
Name: count, dtype: int64
